In [ ]:
pip install spade


# Ejemplo de votacion simple (SI o NO), al final de la votación se sacan los resultados obtenidos en porcentaje

**Recuerda cambiar el nombre de los agentes**

In [ ]:
import spade
from spade.message import Message
from spade.behaviour import CyclicBehaviour, OneShotBehaviour
import asyncio

class VotingAgent(spade.agent.Agent):
    def __init__(self, jid, password, participants):
        super().__init__(jid, password)
        self.participants = participants
        self.votes = {"YES": 0, "NO": 0}

    class CollectVotesBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                vote = msg.body
                self.agent.votes[vote] += 1

    class InitiateVotingBehaviour(OneShotBehaviour):
        async def run(self):
            # Enviar solicitudes de votación
            print("Iniciando votación")
            for participant in self.agent.participants:
                msg = Message(to=participant)
                msg.set_metadata("performative", "request")
                msg.body = "Please vote YES or NO"
                await self.send(msg)

            # Esperar a que los agentes voten
            await asyncio.sleep(20)  # Ajusta este tiempo según sea necesario

            # Calcular y mostrar los resultados
            total_votes = sum(self.agent.votes.values())
            if total_votes > 0:
                yes_percentage = (self.agent.votes["YES"] / total_votes) * 100
                no_percentage = (self.agent.votes["NO"] / total_votes) * 100
                print(f"Results: YES: {yes_percentage:.2f}%, NO: {no_percentage:.2f}%")
            else:
                print("No votes received.")

    async def setup(self):
        self.add_behaviour(self.CollectVotesBehaviour())
        self.add_behaviour(self.InitiateVotingBehaviour())



In [ ]:
import random

class ParticipantAgent(spade.agent.Agent):
    class VotingBehaviour(CyclicBehaviour):
        async def run(self):
            msg = await self.receive(timeout=10)
            if msg:
                vote = random.choice(["YES", "NO"])
                reply = msg.make_reply()
                reply.body = vote
                await self.send(reply)

    async def setup(self):
        self.add_behaviour(self.VotingBehaviour())


In [ ]:

num_participants = 2  # Número de participantes
participant_jids = [f"participant{i}@gtirouter.dsic.upv.es" for i in range(num_participants)]


# Iniciar agentes participantes
participants = [ParticipantAgent(jid, "secret") for jid in participant_jids]
for participant in participants:
    await participant.start()

await asyncio.sleep(5)  # Esperar a que se complete la creacion de los participantes
    
# Iniciar el agente de votación
voting_agent = VotingAgent("voting@gtirouter.dsic.upv.es", "secret", participant_jids)
await voting_agent.start()

await asyncio.sleep(10)  # Esperar a que se complete la votación y se muestren los resultados

# Detener a todos los agentes
await voting_agent.stop()
for participant in participants:
    await participant.stop()

